In [1]:
# run this to shorten the data import from the files
import os
path_data = os.path.join(os.path.dirname(os.getcwd()), 'datasets/')


In [2]:
import pandas as pd 
bikes = pd.read_csv(path_data+'bikes_test.csv')
bikes.head()

,work_day,temp,humidity,wind_speed,num_bikes
0,0,0.265833,0.687917,0.175996,2.947
1,1,0.282609,0.622174,0.153800,3.784
2,1,0.354167,0.496250,0.147379,4.375
3,1,0.256667,0.722917,0.133721,2.802
4,1,0.265000,0.562083,0.194037,3.830


In [3]:
# exercise 01

"""
Sampling posterior draws

Tired of working for the central government and for the marketing company, you take a new job as a data analyst for your city's local authorities. The city operates a bike-sharing system in the city and they ask you to predict the number of bikes rented per day to plan staff and repairs accordingly.

You have been given some data on the number of rented vehicles per day, temperature, humidity, wind speed, and whether the day was a working day:

     work_day      temp  humidity  wind_speed  num_bikes
0           0  0.344167  0.805833    0.160446      0.985
1           0  0.363478  0.696087    0.248539      0.801
..        ...       ...       ...         ...        ...
698         1  0.280870  0.555652    0.115522      5.323
699         1  0.298333  0.649583    0.058471      5.668

Try building a regression model to predict num_bikes using the bikes DataFrame and pymc3 (aliased as pm).

NOTE: Calling pm.sample() for the first time in a fresh Python session takes some time, as Python code is being compiled to C under the hood. To save you time, we only ask you to get the code right instead of executing it.
"""

# Instructions

"""

    Reorder the code lines to sample posterior draws, remembering to indent the lines that need it.

"""

# solution

import pymc3 as pm

formula = 'num_bikes ~ temp + work_day'

with pm.Model() as model_1:
    pm.GLM.from_formula(formula, data=bikes)
    trace_1 = pm.sample(draws=1000, tune=500)

#----------------------------------#

# Conclusion

"""
Good job! In just a couple of lines of code you can sample posterior draws in a Bayesian linear regression model! Let's take a look at what to do with the resulting trace!
"""

/home/nero/Documents/Estudos/estudos/lib/python3.11/site-packages/theano/scalar/basic.py:2323: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  self.ctor = getattr(np, o_type.dtype)


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# exercise 02

"""
Inspecting posterior draws

You continue working on your task to predict the number of bikes rented per day in a bike-sharing system. The posterior draws from your regression model which you sampled before are available in your workspace as trace_1.

You know that after obtaining the posteriors, it is best practice to take a look at them to see if they make sense and if the MCMC process has converged successfully. In this exercise, you will create two plots visualizing posterior draws and summarize them in a table. Let's inspect our posteriors!

NOTE: Please allow up to half a minute for the plots to render, since they have many draws to process.
"""

# Instructions

"""

    Import pymc3 under its usual alias, pm.
    Draw a trace plot of trace_1.
    Draw a forest plot of trace_1.
---
Question

Print the table with posterior draws' summary statistics by passing trace_1 to pm.summary(). Based on this table and the two plots you have just created, which of the following statements is false?

    Given the 94% credible interval, we are not sure whether more bikes are rented in the working week than in the weekends.

    Judging by the trace plot, sampling for all parameters has converged successfully.

    The R-hat statistic signifies some convergence issues. (Answer)
"""

# solution

# Import pymc3
import pymc3 as pm

# Draw a trace plot of trace_1
pm.traceplot(trace_1)
plt.show()

# Draw a forest plot of trace_1
pm.forestplot(trace_1)
plt.show()

#----------------------------------#

# Conclusion

"""
Correct, this one's false! R-hat values above 1 signify convergence issues, and here R-hat is exactly 1 for all parameters, which denotes a successful convergence.
"""

In [ ]:
# exercise 03

"""
Comparing models with WAIC

Now that you have successfully built the first, basic model, you take another look at the data at your disposal. You notice a variable called wind_speed. This could be a great predictor of the numbers of bikes rented! Cycling against the wind is not that much fun, is it?

You fit another model with this additional predictor:

formula = "num_bikes ~ temp + work_day + wind_speed"

with pm.Model() as model_2:
    pm.GLM.from_formula(formula, data=bikes)
    trace_2 = pm.sample(draws=1000, tune=500)

Is your new model_2 better than model_1, the one without wind speed? Compare the two models using Widely Applicable Information Criterion, or WAIC, to find out!

Both trace_1 and trace_2 are available in your workspace, and pycm3 has been imported as pm.
"""

# Instructions

"""

    Create a dictionary traces_dict with two keys, trace_1 and trace_2, holding the corresponding trace objects.
---

    Call the appropriate pymc3 function to create a comparison table based on traces_dict, using waic for comparison, and assign the result to comparison.
---

    Draw a comparison plot between the two models (a textsize argument was added to the plotting function you need to call to improve the plot's readability).
---
Question

You can print() the comparison between the two models you have created in the console. Based on the output, and on the comparison plot you have just drawn, which of the following statements is false?
Possible answers:
    The probability that it is model_2  of the two models that is true is around 2% (answer)
"""

# solution

# Gather trace_1 and trace_2 into a dictionary
traces_dict = {"trace_1": trace_1, "trace_2": trace_2}

# Create a comparison table based on WAIC
comparison = pm.compare(traces_dict, ic="waic")

# Draw a comparison plot
pm.compareplot(comparison, textsize=20)
plt.show()

#----------------------------------#

# Conclusion

"""
Yes, it's the other way around! The weight of 0.021 for model_1 indicates it is the true model with around 2% probability.
"""

In [ ]:
# exercise 04

"""
Sample from predictive density

Finally! Your job is to predict the number of bikes rented per day, and you are almost there. You have fitted the model and verified the quality of parameter draws. You have also chosen the better of the two competing models based on the WAIC. Now, it's time to use your best model to make predictions!

A couple of new observations, not seen by the model, have been collected in a DataFrame named bikes_test. For each of them, we know the true number of bikes rented, which will allow us to evaluate model performance. In this exercise, you will get familiar with the test data and generate predictive draws for every test observation. The trace of your model which you have generated before is available as trace_2, and pymc3 has been imported as pm. Let's make predictions!
"""

# Instructions

"""

    Print the head of bikes_test to the console and get familiar with the data.
---

    Define the model formula to predict num_bikes using temp, work_day and wind_speed as predictors.
---

    Generate predictive draws for the test data and assign the result to posterior_predictive.

"""

# solution

# Print bikes_test head
print(bikes.head())

# Define the formula
formula = "num_bikes ~ temp + work_day + wind_speed"

# Generate predictive draws
with pm.Model() as model:
    pm.GLM.from_formula(formula, data=bikes)
    posterior_predictive = pm.fast_sample_posterior_predictive(trace_2)

#----------------------------------#

# Conclusion

"""
Well done generating predictive draws! Let's proceed to the next exercise to find out how good your predictions are!
"""

In [ ]:
# exercise 05

"""
Estimating test error

Now that you have your posterior_predictive (available to you in your workspace), you can evaluate model performance on new data. To do this, you will need to loop over the test observations, and for each of them, compute the prediction error as the difference between the predictive distribution for this observation and the actual, true value. This will give you the distribution of your model's error, which you can then visualize.

You will need pymc3 and numpy, which have been imported for you as pm and np, respectively. The test data, bikes_test, is also available in your workspace. Let's get to it!
"""

# Instructions

"""

    Initialize errors as an empty list.
    For each row in bikes_test, calculate prediction error as the predictive draws for this row from posterior_predictive minus the single true value of num_bikes from the row.
    Reshape errors by converting them to a numpy array and applying the .reshape() method to the outcome, and assign the final result to error_distribution.
    Plot the test error distribution using pymc3's plot_posterior() function.

"""

# solution

# Initialize errors
errors = []

# Iterate over rows of bikes_test to compute error per row
for index, test_example in bikes_test.iterrows():
    error = posterior_predictive['y'][:, index] - test_example['num_bikes']
    errors.append(error)

# Reshape errors
error_distribution = np.array(errors).reshape(-1)

# Plot the error distribution
pm.plot_posterior(error_distribution)
plt.show()

#----------------------------------#

# Conclusion

"""
Outstanding job! This was a tough one! In practice, you might want to compute the error estimate based on more than just 10 observations, but you can already see some patterns. For example, the error is more often positive than negative, which means that the model tends to overpredict the number of bikes rented!
"""

In [ ]:
# exercise 06

"""
Fitting the model

You can use a linear regression model to estimate the avocado price elasticity. The regression formula should be:

Here,

will be the price elasticity, that is the impact of price on sales. You will assume that the elasticity is the same for regular and organic avocados. You also expect it to be negative: the higher the price, the lower the sales, that's the case for most goods. To incorporate this prior knowledge into the model, you decide to use a normal distribution with mean -80 as the prior for price. How would you build such a model?

NOTE: Recall that calling pm.sample() for the first time in a fresh Python session takes some time, as Python code is being compiled to C under the hood. To save you time, we only ask you to get the code right instead of executing it.
"""

# Instructions

"""

    Reorder the code lines to sample posterior draws, remembering to indent the lines that need it.

"""

# solution

formula = 'volume ~ price + type_organic'

with pm.Model as model:
    priors = {'price' : pm.Normal.dist(mu=-80)}
    pm.GLM.from_formula(formula, data=avocado, priors=priors)
    trace = pm.sample(draws=1000, tune=500)

#----------------------------------#

# Conclusion

"""
Well done! That's how you would fit the model and generate posterior parameter draws! Let's inspect the model in the next exercise.
"""

In [ ]:
# exercise 07

"""
Inspecting the model

Well done getting the model-building right! The trace is available in your workspace and, following the best practices, you will now inspect the posterior draws to see if there are any convergence issues. Next, you will extract each model parameter from the trace and summarize it with its posterior mean. These posterior means will come in handy later, when you will be making predictions with the model. Let's take a look at the parameter draws!

You will need to use pymc3 and numpy, which have been imported for you as pm and np, respectively.

NOTE: Please allow up to half a minute for the plots to render, since they have many draws to process.
"""

# Instructions

"""

    Draw and examine the trace plot of posterior draws.
    Create a summary of posterior draws, assign in to summary, and print it.
---

    Calculate posterior mean of each parameter by extracting draws from trace and wrapping them with a function computing the mean.
    Assign the posterior means to intercept_mean, organic_mean, price_mean, and sd_mean, accordingly.

"""

# solution

# Draw a trace plot of trace
pm.traceplot(trace)
plt.show()

# Print a summary of trace
summary = pm.summary(trace)
print(summary)

# Extract each model parameter
intercept_mean = np.mean(trace.get_values("Intercept")) 
organic_mean = np.mean(trace.get_values("type_organic")) 
price_mean = np.mean(trace.get_values("price")) 
sd_mean = np.mean(trace.get_values("sd")) 

#----------------------------------#

# Conclusion

"""
Well done! Have you noticed something unusual when it comes it MCMC convergence? Look at the left part of the trace plot for price: the density of one of the chains is slightly wobbly. Luckily, it's only one chain and its density is still quite close to the densities of other chains. So, all in all, we don't need to worry about it and we can safely use the model to optimize the price!
"""

In [ ]:
# exercise 08

"""
Optimizing the price

Great job on fitting and inspecting the model! Now, down to business: your boss asks you to provide the avocado price that would yield the largest profit, and to state what profit can be expected. Also, they want the price to be divisible by $0.25 so that the customers can easily pay with quarters.

In this exercise, you will use your model to predict the volume and the profit for a couple of sensible prices. Next, you will visualize the predictive distributions to pick the optimal price. Finally, you will compute the credible interval for your profit prediction. Now go and optimize!

The posterior means you have computed before are available to you as intercept_mean, organic_mean, price_mean, and sd_mean, respectively. Also, pymc3, arviz, and numpy are imported as pm, az, and np.
"""

# Instructions

"""

    For each price in 0.5, 0.75, 1 and 1.25, calculate the predictive mean.
    Sample from the predictive distribution to predict sales volume.
    Use the predicted volume to predict the profit.
---

    Draw a forest plot of predicted profit.
---

    Based on the plot you have just created, pick the optimal price and calculate the Highest Posterior Density credible interval of 99% for this price.

"""

# solution

# For each price, predict volume and use it to predict profit
predicted_profit_per_price = {}
for price in [0.5, 0.75, 1, 1.25]:
    pred_mean = (intercept_mean + price_mean * price + organic_mean)
    volume_pred = np.random.normal(pred_mean, sd_mean, size=1000)
    profit_pred = price * volume_pred
    predicted_profit_per_price.update({price: profit_pred})
    
# Draw a forest plot of predicted profit for all prices
pm.forestplot(predicted_profit_per_price)
plt.show()

# Calculate and print HPD of predicted profit for the optimal price
opt_hpd = az.hdi(predicted_profit_per_price[0.75], credible_interval=0.99)
print(opt_hpd)

#----------------------------------#

# Conclusion

"""
Terrfic work! With a higher or lower price, your company would lose profit, but thanks to your modeling skills, they were able to set the best possible price. More than that, knowing the uncertainty in the profit prediction, they can prepare for the worst-case scenario (in which the profit is negative)! 
"""